# 1. 프레임, 박스좌표 => csv 파일로 저장

In [1]:
# 모듈 로딩
import cv2
from yolov5facedetector.face_detector import YoloDetector


url='../asset/input/test_01.mp4'


def mosaic(url, yolo_type='yolov5n',  target_size=480, gpu=0, min_face=0, conf_thres=0.3, iou_thres=0.5):
    model = YoloDetector(yolo_type=yolo_type, target_size=target_size, gpu=gpu, min_face=min_face) 
    webcam = cv2.VideoCapture(url)


    # 동영상 파일 열기 성공 여부 확인
    if not webcam.isOpened():
        print("Could not open webcam") 
        exit()
        
    while webcam.isOpened():
        
        status, frame = webcam.read()
        
        if frame is None: break
        
        frame_value.append(frame)

        bboxes, confs, points = model.predict(frame, conf_thres=conf_thres, iou_thres=iou_thres)
    if val_csv == 0:

    else :
        face_box.append(bboxes)
       

frame_value = []   # 프레임 담을 리스트
face_box = []   # 얼굴 박스 좌표 담을 리스트

mosaic(url)

In [2]:
import pandas as pd

df = pd.DataFrame({'프레임':frame_value, '얼굴박스좌표':face_box})

df

,프레임,얼굴박스좌표
0,"[[[0, 0, 4], [1, 0, 11], [0, 0, 10], [0, 0, 9]...","[[[518, 92, 691, 313]]]"
1,"[[[0, 0, 4], [1, 0, 11], [0, 0, 10], [0, 0, 9]...","[[[517, 91, 689, 314]]]"
2,"[[[0, 0, 4], [1, 0, 11], [0, 0, 10], [0, 0, 9]...","[[[516, 90, 687, 314]]]"
3,"[[[0, 0, 4], [1, 0, 11], [0, 0, 10], [0, 0, 9]...","[[[515, 95, 685, 318]]]"
4,"[[[0, 0, 4], [1, 0, 11], [0, 0, 10], [0, 0, 9]...","[[[514, 98, 683, 322]]]"
...,...,...
197,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...","[[[421, 119, 726, 530]]]"
198,"[[[2, 0, 0], [2, 0, 0], [2, 0, 0], [2, 0, 0], ...","[[[425, 117, 729, 529]]]"
199,"[[[2, 0, 0], [2, 0, 0], [2, 0, 0], [2, 0, 0], ...","[[[428, 112, 736, 528]]]"
200,"[[[2, 0, 0], [2, 0, 0], [2, 0, 0], [2, 0, 0], ...","[[[433, 108, 743, 527]]]"


## 2. 실시간 모자이크 처리 영상 속도 => 원본 속도랑 비슷하게 하는 코드

### 모자이크 영상 처리 상태 시각화

In [1]:
'''
sigmaX 높을수록 뿌옇게 변함(비식별화 잘 됨)
'''

# 모듈 로딩
import cv2
from yolov5facedetector.face_detector import YoloDetector
import time
import math
import re

url = './data/people.mp4'


def video_save(webcam, file_name):
    '''
    영상 저장을 위한 객체 생성
    '''
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    width = round(webcam.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = round(webcam.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = webcam.get(cv2.CAP_PROP_FPS)
    out = cv2.VideoWriter(file_name, fourcc, fps, (width, height))  # 파일명 경로
    return fps, out


def mosaic(url, yolo_type='yolov5n',  target_size=480, gpu=0, min_face=0, conf_thres=0.3, iou_thres=0.5, sigma=10):
    model = YoloDetector(yolo_type=yolo_type, target_size=target_size, gpu=gpu, min_face=min_face) 
    webcam = cv2.VideoCapture(url)
    fps, out = video_save(webcam, f"{re.sub('.mp4','',url.split('//')[0])}_mosaic.mp4")

    # 동영상 파일 열기 성공 여부 확인
    if not webcam.isOpened():
        print("Could not open webcam") 
        exit()
        
    # mosa=True일 때 영상의 기본 상태 : 모자이크
    mosa = True

    fps_value = 1./fps  # 원본 fps와 맞추기 위해 나눌 수

    while webcam.isOpened():

        status, frame = webcam.read()

        # 알고리즘 시작 지점
        start_time = time.time()

        if frame is None: break

        if (webcam.get(cv2.CAP_PROP_POS_FRAMES) == 1) | (webcam.get(cv2.CAP_PROP_POS_FRAMES) == 2) | (webcam.get(cv2.CAP_PROP_POS_FRAMES) % fps_value == 0):
            bboxes, confs, points = model.predict(frame, conf_thres=conf_thres, iou_thres=iou_thres)
        
        else: pass

        if not status:
            print("Could not read frame")
            exit()

        key = cv2.waitKey(1)
        if key == 26:  # Ctrl + Z : 모자이크 켜짐
            mosa = True
        elif key == 24:  # Ctrl + X : 모자이크 꺼짐
            mosa = False

        for bbox in bboxes[0]:
            (startX, startY)=bbox[0], bbox[1]
            (endX, endY)=bbox[2], bbox[3]

            if mosa == True:
                face_region = frame[startY:endY, startX:endX]  # 관심영역(얼굴) 지정
                
                # 모자이크
                '''
                cv2.GaussianBlur(src, ksize, sigmaX, dst=None, sigmaY=None, borderType=None) -> dst

                • src: 입력 영상. 각 채널 별로 처리됨.
                • dst: 출력 영상. src와 같은 크기, 같은 타입.
                • ksize: 가우시안 커널 크기. (0, 0)을 지정하면 sigma 값에 의해 자동 결정됨
                • sigmaX: x방향 sigma.
                • sigmaY: y방향 sigma. 0이면 sigmaX와 같게 설정.
                • borderType: 가장자리 픽셀 확장 방식.
                '''
                frame[startY:endY, startX:endX] = cv2.GaussianBlur(face_region, ksize=(0,0), sigmaX=sigma)


        
        # display output

        out.write(frame)   # 동영상 저장

        cv2.imshow("Mosaic Video", frame)  # 윈도우 창에 이미지를 띄움

        # 알고리즘 종료 시점
        # print('모자이크 처리에 걸리는 시간 \n▶ FPS', int(1./(time.time() - start_time)),\
        #     '\n▶ Time:',  time.time() - start_time, '\n')
        
        if webcam.get(cv2.CAP_PROP_POS_FRAMES) == 2:
            fps_value = math.ceil((time.time() - start_time)/(1./fps))

        if cv2.waitKey(1) & 0xFF == ord('q'):  # 'q' 키 입력 받으면 윈도우 창이 종료
            break

        print(' '*round(webcam.get(cv2.CAP_PROP_POS_FRAMES)*100/webcam.get(cv2.CAP_PROP_FRAME_COUNT))+'▽',\
            round(webcam.get(cv2.CAP_PROP_POS_FRAMES)*100/webcam.get(cv2.CAP_PROP_FRAME_COUNT)),'%')
        print('[', end='')
        print(round(webcam.get(cv2.CAP_PROP_POS_FRAMES)*100/webcam.get(cv2.CAP_PROP_FRAME_COUNT))*'■'+\
            (100-round(webcam.get(cv2.CAP_PROP_POS_FRAMES)*100/webcam.get(cv2.CAP_PROP_FRAME_COUNT)))*' ', end='')
        print(']')
        

    #동영상 파일을 닫고 메모리 해제
    out.release()  
    webcam.release()  


    # 모든 윈도우 창을 닫음
    cv2.destroyAllWindows() 

frame_value = []   # 프레임 담을 리스트
face_box = []   # 얼굴 박스 좌표 담을 리스트

mosaic(url)

C:\Users\USER\AppData\Roaming\Python\Python39\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


▽ 0 %
[                                                                                                    ]
 ▽ 1 %
[■                                                                                                   ]
 ▽ 1 %
[■                                                                                                   ]
  ▽ 2 %
[■■                                                                                                  ]
  ▽ 2 %
[■■                                                                                                  ]
  ▽ 2 %
[■■                                                                                                  ]
   ▽ 3 %
[■■■                                                                                                 ]
   ▽ 3 %
[■■■                                                                                                 ]
    ▽ 4 %
[■■■■                                                                                                ]
 

## 3. 모자이크 처리 영상 용량 줄이기

### https://codealone.tistory.com/80

In [11]:
'''
crf 높을수록 선명하지만, 용량 높아짐
'''
# 모듈 로딩
import ffmpeg
import os

def compact_save(PATH='./data/', crf=25):
    '''
    모자이크 영상 용량 줄이는 함수
    '''
    
    # ffmpeg으로 변환할 때는 기존 파일에 덮어쓰기가 안되므로 변환된 파일들을 저장할 폴더를 생성
    os.mkdir('resized')
    
    # 폴더 내의 파일 목록을 모두 가져온다.
    files = os.listdir(PATH)
    
    # 파일 목록 중에 '_mosaic.mp4' 파일만 추출
    mp4s = [file for file in files if file[-11:] == '_mosaic.mp4']

    # print(mp4s)

    # 변환 중 에러가 발생하는 경우가 있어 에러 처리를 위한 리스트를 만듦
    err = []

    for mp4 in mp4s:

        # (try) 파일 변환을 시도함.
        # (except) 에러가 날 경우 변환 중이던 파일을 지우고 파일명을 err 리스트에 추가함

        try:
            # crf[quality] : 비트레이트 대신 화질 기준으로 인코딩할 때 쓰는 옵션. libx264 코덱 기준 사용 가능 범위 0-51, 0은 무손실, 디폴트는 23
            ffmpeg.input(PATH+mp4).output('resized/'+mp4, crf=crf, vsync='vfr').run()

        except:
            os.remove('resized/'+mp4)
            err.append(mp4)

        # 파일이 resized 폴더 내에 있다면(에러가 나지 않고 변환되어 옮겨져 있다면)
        # 원래 폴더에서 파일을 삭제함
        if mp4 in os.listdir('resized'):
            os.remove(PATH+mp4)

    # mp4 파일 목록을 순화하면서 오류 났던 파일 (err 리스트)을
    # 제외한 나머지 파일을 원래 폴더로 모두 옮김
    for mp4 in mp4s:
        if mp4 in err:
            pass
        else:
            os.replace('resized/'+mp4, PATH+mp4)

    # resized 폴더를 삭제함
    os.rmdir('resized')

    # 변환에 실패한 파일 목록을 출력함
    # print('실패한 목록: \n', err)

compact_save()